# Project
## Link to github repository
https://github.com/sunxinyan79214219/my-dcsi-project-Use-version/blob/main/new%20use%20homework%20project%201.ipynb


Predicting Newsletter Subscriptions: A Deep Dive into Player Behaviors and Demographics Within a Minecraft Server Ecosystem


### Introduction

## Background


In the dynamic landscape of the video game industry, deciphering player behavior stands as a cornerstone for crafting immersive experiences and nurturing enduring player relationships. Game - related newsletters serve as a vital conduit for developers to stay connected with their audience, delivering a mix of updates, promotions, and community spotlights. Yet, the decision to subscribe to these newsletters varies among players. Pinpointing the characteristics and behaviors most strongly linked to subscription empowers developers to tailor communication strategies and enhance outreach efforts.
This report leverages player data from a Minecraft server environment. It delves into patterns governing newsletter subscriptions, drawing on variables like gameplay habits (such as the frequency of sessions), demographic factors (including age), and experience levels.

Data Description
The dataset includes anonymized player-level information from a Minecraft multiplayer server, combining gameplay activity, demographic data, and newsletter subscription status.

Question:For the Question I choose, I choose Question1, What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, 
and how do these features differ between various player types?

Data Description

Number of observations (players): 196

Number of variables: Initially 9, refined to 4 after data cleaning

Source: Gathered via user accounts and server logs

Data collection method: Derived from gameplay logs and demographic information submitted by users

Response variable: subscribe – indicates if a player subscribed to the newsletter (taking values of TRUE or FALSE)

# players.csv: A list of all unique players, including data about each player.

- **Variable summary**:
| Column Name     | Description                                | Type     |
|------------------|--------------------------------------------|----------|
| `hashedEmail`    | Unique ID for each player                  | Categorical   |
| `subscribe`      | Whether the player subscribed to the newsletter (True or False) | Logical  |
| `experience`     | Player experience level (e.g., Pro, Amateur) | Categorical   |
| `gender`         | Player's gender                            | Categorical   |
| `Age`            | Player's age                               | Numeric  |
| `played_hours`   | Total hours the player spent in the game   | Numetric   |
| `name`           | Player’s in-game name (not used here)      | Categorical   |
| `hashedEmail`        | Unique ID to match with `players.csv`       | Categorical   |
| `original_start_time`| Start time of the session (Unix timestamp) | Numeric   |
| `original_end_time`  | End time of the session (Unix timestamp)   | Numeric   |

# session.csv: A list of individual play sessions by each player, including data about the session.

- **Variable summary**:
| Column Name     | Description                                | Type     |
|------------------|--------------------------------------------|----------|
| `hashedEmail`    | Unique player identifier (matches players.csv)               | Categorical   |
| `start_time`      | Session start time (DD/MM/YYYY HH:MM) | Datetime  |
| `end_time`     | Session end time (DD/MM/YYYY HH:MM) | Datetime   |
| `original_start_time`         | Session start time (Unix timestamp in scientific notation)                           | Numeric   |
| `original_end_time`            |Session end time (Unix timestamp in scientific notation)                              | Numeric  |















## Key characteristics of the data:

Multiple sessions per player (panel/longitudinal data)

Time-stamped behavioral data

Hashed email addresses for privacy

Potentially large number of observations (based on sample data)










# Methods & Results

# Step 1: Load and Prepare the Data from players.csv and sessions.csv
Firstly, I should loaded the necessary libraries to support data wrangling, visualization, and modeling.

Then, I used read_csv() to import the two provided CSV files: one containing player information and the other containing session data.


In [23]:
library(tidyverse)
library(repr)
library(tidymodels)
players <- read_csv("players.csv")
sessions <- read_csv("sessions.csv")
library(dplyr)


Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [28]:
library(tidyverse)
library(repr)
library(tidymodels)
players <- read_csv("players.csv")
sessions <- read_csv("sessions.csv")
library(dplyr)
# first, counts Calculate the number of sessions and merge data

session_counts <- sessions|>
  group_by(hashedEmail) |> 
  summarise(num_sessions = n())

players <- players |>
  left_join(session_counts, by = "hashedEmail") |>
  filter(!is.na(num_sessions) & !is.na(Age))

# Create age groups (≥ 18 vs<18)
players <- players |>
  mutate(age_group = ifelse(Age >= 18, "≥18", "<18"))
players

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age,num_sessions,age_group
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<chr>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,27,<18
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17,3,<18
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17,1,<18
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21,1,≥18
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21,1,≥18
Amateur,FALSE,1d2371d8a35c8831034b25bda8764539ab7db0f63938696917c447128a2540dd,0.0,Emerson,Male,21,1,≥18
Amateur,TRUE,8b71f4d66a38389b7528bb38ba6eb71157733df7d1740371852a797ae97d82d1,0.1,Natalie,Male,17,1,<18
Veteran,TRUE,7a4686586d290c67179275c7c3dfb4ea02f4d317d9ee0e2cee98baa27877a875,1.6,Lane,Female,23,3,≥18
Beginner,TRUE,4bed6f49b672974f706aa7f020b4f29020326bf594520593557cd381a51559b8,1.5,Sarah,Female,25,1,≥18


In [29]:
# Draw a bar chart of age groups and subscription status

age_subscribe <- ggplot(players, aes(x = age_group, fill = subscribe)) +
  geom_bar(position = "dodge") +
  labs(title = "Subscription Rate by Age Group", 
       x = "Age Group", y = "Count") +
  theme_minimal()

# Draw a box plot of age groups and number of conversations
age_sessions <- ggplot(players, aes(x = age_group, y = num_sessions, fill = age_group)) +
  geom_boxplot() +
  labs(title = "Session Count by Age Group", 
       x = "Age Group", y = "Number of Sessions") +
  theme_minimal()